This script applys the LASSO filter to downloaded Sentinel-1 files, saving them in a folder 's1_gf' for each site.

The site folders containing the raw tifs should be saved under "C:/Users/micha/Documents/Pod_sites_data/S1/" (or change variable below).

The 's1_gf' folders containing the output images are created in each project folder.

In [ ]:
import cv2
import numpy as np


def boxfilter(I, r):
    """
    Apply box filter on the input image
    Params:
        I -- (narray) input image
        r -- (int) kernel size
    Returns:
        (narray)
    """
    bf = cv2.boxFilter(I, -1, (r, r))
    return bf


def guidefilter(I, P, r, eps):
    """
    This function applies guided filter to the input image P using a guide Image I.
    Params:
        I -- (narray) Guide image
        P -- (narray) Image to be filted
        r -- (int) Kernel size
        eps -- (float) Regularization coefficient
    Returns:
        (narray)
    """

    N = boxfilter(np.ones(np.shape(I)), r)  # size of each local patch

    mean_I = boxfilter(I, r) / N
    mean_P = boxfilter(P, r) / N
    mean_IP = boxfilter(I * P, r) / N
    cov_IP = mean_IP - mean_I * mean_P  # covariance of (I,P) in each local patch

    mean_II = boxfilter(I * I, r) / N
    var_I = mean_II - mean_I * mean_I

    a = cov_IP / (var_I + eps)  # equation 5 in the paper
    b = mean_P - a * mean_I  # equation 6 in the paper

    mean_a = boxfilter(a, r) / N
    mean_b = boxfilter(b, r) / N

    q = mean_a * I + mean_b  # equation 8 in the paper
    return q



def guidefilter_ite(img, r_start, eps_start, ite):
    """
    Apply guided filter in specified number of iterations, for each of which the kernel size increases while eps
    decreases.
    Params:
        img -- (narrray) Input data for filtering
        r_start -- (int) Kernel size in first iteration
        eps_start - (float) Regularization coefficient in first iteration
        ite -- (int) iteration
    Returns:
        (narray)
    """
    r = r_start
    eps = eps_start

    for i in range(ite):
        eps /= 3 ** i
        img = guidefilter(img, img, r, eps)
        r += 2

    return img


def guidefilter_ite2(img, r_start, eps_start, ite):

    for i in range(2):
        img = guidefilter_ite(img, r_start, eps_start, ite)

    return img

In [ ]:
! pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.2 MB/s eta 0:00:00


In [ ]:
import os
import yaml
import pickle
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np
#from guided_filter import guidefilter_ite2, guidefilter


def preprocess(file, dir_img, dir_out, config="./config.yaml"):
    """
    This function applies preprocess of resampling and filtering on each downloaded raw Sentinel file. The preprocessed
    image is saved as an .npy file for efficient usage in regression.
    Params:
        file -- (str) File name of image to be resampled
        config -- (str) A yaml file for configuration. The key named "Preprocess" is used in this function.
    """
    with open(config, "r") as config:
        params = yaml.safe_load(config)["Lasso"]

    aoi = params["aoi"]
    polarizations = params["polarizations"]

  #  dir_img = params["dir_img"].format(aoi)
    dir_ready = os.path.split(params["dir_ready"])[0].format(aoi)

    #dir_meta = params["dir_meta"].format(dir_ready, aoi)

    # Filter parameters
    r = params["kernel"]
    eps = params["eps"]
    ite = params["iteration"]

    # dst metadata
#     with open(dir_meta, "rb") as dst:
#         dst_meta = pickle.load(dst)
#         dst_canvas = np.zeros((dst_meta["height"], dst_meta["width"]), dtype=np.float64)
#         dst_transform = dst_meta["transform"]
#         dst_crs = dst_meta["crs"]


    for i in range(len(polarizations)):

        with rasterio.open(os.path.join(dir_img, file)) as src:
            img = src.read(i + 1)
            src_transform = src.transform
            src_crs = src.crs

        # Filter
        print("start filtering")
        print("polarization", polarizations[i])
        I = np.where(np.isnan(img), -99, img)
        print("I", img[~np.isnan(img)])
        print(type(img))
        print(type(I))
        print('I', img.shape)


        out_img = guidefilter_ite2(I, r, eps, ite)
        # out_img = guidefilter(I, I, r, eps)
        out_img = np.where(np.isnan(img), img, out_img)

        print('out_img', out_img[~np.isnan(out_img)])
        del I, img

        # # Write preprocessed images into narray
        polarization = polarizations[i]
      #  dir_out = params["dir_ready"].format(aoi, aoi, polarization)
        if not os.path.isdir(dir_out):
            os.mkdir(dir_out)

        out_name = "{}_{}.npy".format(file.split(".")[0], polarization)
        out_name = "{}_{}.tif".format(file.split(".")[0], polarization)

       # np.save(os.path.join(dir_out.format(aoi, polarization), out_name),
        #        out_img)

        print('out_img', out_img.shape)

        new_dataset = rasterio.open(os.path.join(dir_out.format(aoi, polarization), out_name),
                                    'w', driver='GTiff',  height=out_img.shape[0],
                                    width=out_img.shape[1],  count=1,  dtype=out_img.dtype,  crs= src_crs,
                                    transform= src_transform )
        new_dataset.write(out_img, 1)
#         np.save(os.path.join(dir_out.format(aoi, polarization), out_name),
#                 reproject(
#                     source=out_img,
#                     destination=dst_canvas,
#                     src_transform=src_transform,
#                     src_crs=src_crs,
#                     dst_transform=dst_transform,
#                     dst_crs=dst_crs,
#                     resampling=Resampling.bilinear)[0]
#                 )
    print("finish: ", out_name)

In [ ]:
import glob

def preprocess_folder(folder):
    config = "C:/Users/micha/Documents/mcecil_milestones/scripts/python_notebooks/config.yaml"
    folder_out = folder + "/s1_gf"
    #print(folder_out)
    #files = os.listdir(folder)
    files = [f for f in os.listdir(folder) if f.endswith('.tif')]
    print(files)

    os.chdir(folder)
    for file in files:
        preprocess(file, folder, folder_out, config)



In [ ]:
root_folder = "C:/Users/micha/Documents/Pod_sites_data/S1/"
site_name_array = os.listdir(root_folder)

print(root_folder + site_name_array[1])


In [ ]:

for site in site_name_array:
    print(site)
    preprocess_folder(root_folder + site)